# Cleaning the dataset, preparing for analisis.

In [306]:
import pandas as pd
import numpy as np
columns = ["oferta","propiedad","monto","moneda","direccion", \
    "inmobiliaria","comentario","metros_quad","n_dormitorios","n_baños"]
raw = pd.read_csv("../data/real_estate_data_raw.csv")
raw.columns = columns

print(len(raw))

raw = raw.drop_duplicates()
raw.dropna(subset=["monto", "moneda", "direccion", "metros_quad"]) # dropping rows that lack information in this vital subset
raw.reset_index(inplace=True)
print(len(raw))


33686
33156


### Preparing column "monto", the price of every propiety

In [307]:

mask= ["." in monto for monto in raw["monto"]]
raw.loc[mask, "monto"] = raw.loc[mask, "monto"].apply(lambda x: x.replace(".", "")) # removing dots that denote thousands
mask = [not number.isnumeric() for number in raw["monto"]] # rows to modifie, that have not numeric value in 'monto' attribute
raw.loc[mask,"monto"] = pd.to_numeric(raw.loc[mask,"monto"]) # modifiing types
#mask = raw.index[mask]
#raw = raw.drop(index=mask)
#raw.reset_index()


print(raw)
print(raw.describe())
raw = raw.convert_dtypes()
print(raw.dtypes)

       index             oferta        propiedad   monto moneda  \
0          0              venta    departamentos    4990     uf   
1          1              venta    departamentos    4490     uf   
2          2              venta    departamentos    3065     uf   
3          3              venta    departamentos    1925     uf   
4          4              venta    departamentos    2531     uf   
...      ...                ...              ...     ...    ...   
33151  33681  arriendo temporal  otros inmuebles   35000      $   
33152  33682  arriendo temporal  otros inmuebles   10000      $   
33153  33683  arriendo temporal  otros inmuebles  230000      $   
33154  33684  arriendo temporal  otros inmuebles  380000      $   
33155  33685  arriendo temporal  otros inmuebles   75000      $   

                                               direccion  \
0       comandante malbec 13133, la dehesa, lo barnechea   
1      josé manuel infante 1045, barrio italia, provi...   
2            ed

### Preparing column "direccion", its adress.

In [308]:
def corrected(comuna):
    if comuna == "tiltil": return "til til"
    elif comuna == "calera": return "la calera"
    elif comuna == "río ibánez": return "río ibáñez"
    elif comuna == "coihaique": return "coyhaique"
    elif comuna == "llaillay": return "llay-llay"
    elif comuna == "paiguano": return "paihuano"
    else: return comuna
chile_reg = pd.read_csv("./Data/communes_of_chile.csv")
raw.loc[:,"comuna"] = raw.loc[:,"direccion"].apply(lambda x: x.split(",")[-1].strip())
mask = [not comuna in chile_reg["comuna"].to_numpy() for comuna in raw["comuna"]]
raw.loc[mask,"comuna"] = raw.loc[mask,"direccion"].apply(lambda x: x.split(",")[0].strip())
raw.loc[mask,"comuna"] = raw.loc[mask,"comuna"].apply(corrected)
mask = [not comuna in chile_reg["comuna"].to_numpy() for comuna in raw["comuna"]]
raw.loc[mask,"comuna"] = raw.loc[mask,"direccion"].apply(lambda x: x.split(",")[-1].strip())
raw.loc[mask,"comuna"] = raw.loc[mask,"comuna"].apply(corrected)
mask = [not comuna in chile_reg["comuna"].to_numpy() for comuna in raw["comuna"]]
raw.loc[mask,"direccion"]



Series([], Name: direccion, dtype: string)

### Preparing column "metros_quad", its area.

Converting by brute force every "metros_quad" column's elements. Also, spliting the column (in a list) by " ".

In [309]:
not_none_m = [not np.any([m is None, m is np.nan, m is pd.NA]) for m in raw.loc[:,"metros_quad"]]
raw = raw.convert_dtypes()
#raw.loc[not_none_mask, "metros_quad"] = raw.loc[not_none_mask, "metros_quad"].apply(lambda x: str(x))
raw.loc[:, "metros_quad"] = raw.loc[:,"metros_quad"].str.split(pat=" ", expand=False)
raw.loc[2568, "metros_quad"]

<NA>

Search for missplaced "n_dormitorio" values in "metros_quad". Move every misplaced element to its righteous column, "n_dormitorios".

In [310]:
rooms_mask = raw.loc[not_none_m, "metros_quad"].apply(lambda x: "dormitorio" in x[-1]) \
    .reindex(index=list(range(len(raw))), fill_value=False)
none_mask_d = [np.any([d is None, d is np.nan, d is pd.NA]) for d in raw.loc[:,"n_dormitorios"]]
#raw.loc[list_logical('and',notnone_mask,rooms_mask), ["metros_quad", "n_dormitorios"]]
raw.loc[np.all([rooms_mask,none_mask_d],axis=0), "n_dormitorios"] = [" ".join(rooms).strip() for rooms in raw.loc[rooms_mask, "metros_quad"]]
raw.loc[np.all([rooms_mask,none_mask_d],axis=0), "metros_quad"] = pd.NA

In [311]:
none_m = [np.any([m is None, m is np.nan, m is pd.NA]) for m in raw.loc[:,"metros_quad"]]
raw.loc[none_m, "metros_quad"]

2568     <NA>
3638     <NA>
3897     <NA>
3939     <NA>
4109     <NA>
         ... 
32820    <NA>
32830    <NA>
32903    <NA>
32978    <NA>
32991    <NA>
Name: metros_quad, Length: 1254, dtype: object

In [312]:
not_none_m= [not r for r in none_m]
wrongs= raw.loc[not_none_m, "metros_quad"].apply(lambda x: "útiles" not in x[-1].strip() and "totales" not in x[-1].strip()) \
    .reindex(index=list(range(len(raw))), fill_value=False) # zero values, nothing else to do
rights = np.all([[not w for w in wrongs],not_none_m], axis=0)
raw.loc[rights, "unidad_medida"] = raw.loc[rights, "metros_quad"].apply(lambda x: x[-2])
raw.loc[rights, "area"] = raw.loc[rights, "metros_quad"].apply(lambda x: x[:-2])
raw.loc[rights, "area"] = raw.loc[rights, "area"].apply(lambda x: [x[0],x[-1]] if len(x)==3 else x)

### Preparing "n_formitorios" and "n_baños"

In this section, I'll prepare the last two columns "n_dormitorios" and "n_baños" without much introduction and utilizing the same method of the previous columns.

In [313]:
not_none_b = [not np.any([b is None, b is np.nan, b is pd.NA]) for b in raw["n_baños"]]
rights_b = raw.loc[:, "n_baños"].apply(lambda x: isinstance(x, str))
wrongs = [not r for r in rights_b]
raw.loc[np.all([wrongs, not_none_b], axis=0), "n_baños"] # zero values, there isn't a value that is not string and is not a void value
raw.loc[np.all([rights_b, not_none_b], axis=0), "n_baños"].apply(lambda x: "baño" not in x).sum() # zero values, there isn't a value that is not string and is not a void value.
raw.loc[:, "n_baños"] = raw.loc[:, "n_baños"].str.split(pat=" ")


In [314]:

raw.loc[rights_b, "n_baños"] = raw.loc[rights_b, "n_baños"].apply(lambda x: x[:-1])
raw.loc[rights_b, "n_baños"] = raw.loc[rights_b, "n_baños"].apply(lambda x: [x[0],x[-1]] if len(x)==3 else x)

In [315]:
not_none_d = [not np.any([d is None, d is np.nan,d is pd.NA]) for d in raw["n_dormitorios"]]
rights_d = raw.loc[:, "n_dormitorios"].apply(lambda x: isinstance(x, str))
wrongs = [not r for r in rights_d]
raw.loc[np.all([wrongs, not_none_d], axis=0), "n_dormitorios"] # zero values, there isn't a value that is not string and is not a void value.
real_wrongs = raw.loc[np.all([rights_d, not_none_d], axis=0), "n_dormitorios"].apply(lambda x: "dormitorio" not in x) \
    .reindex(index=list(range(len(raw))), fill_value=False) # 48 string values that may not belong to this column, 
    # a lot of the instead of the word 'dormitorio' reads privado(s), the rest are 'baño(s)' so I'll be discarting just the later.
raw.loc[real_wrongs, "n_dormitorios"]

225      1 - 15 privados
806       1 - 3 privados
926            1 privado
932       1 - 4 privados
1100           1 privado
1111           1 privado
2049      1 - 2 privados
2111           1 privado
7969         1 - 2 baños
7970              1 baño
7971      1 - 3 privados
7972      1 - 3 privados
7973      1 - 4 privados
7974      1 - 3 privados
7975      1 - 2 privados
7976             2 baños
7977      1 - 2 privados
7978      1 - 2 privados
7979      1 - 3 privados
7980           1 privado
7981              1 baño
7982      1 - 3 privados
7983         1 - 2 baños
7984         1 - 2 baños
7985      1 - 3 privados
7986              1 baño
7988      1 - 3 privados
7989      1 - 3 privados
7991           1 privado
7992      1 - 4 privados
7993           1 privado
7994              1 baño
7995      1 - 2 privados
7996      1 - 3 privados
7997      1 - 3 privados
7998           1 privado
8001           1 privado
8002      1 - 2 privados
8003              1 baño
8004      1 - 3 privados


In [316]:
wrong_bath = raw.loc[np.all([rights_d, not_none_d], axis=0), "n_dormitorios"].apply(lambda x: "baño" in x) \
    .reindex(index=list(range(len(raw))), fill_value=False)
raw.loc[wrong_bath, "n_dormitorios"]

7969      1 - 2 baños
7970           1 baño
7976          2 baños
7981           1 baño
7983      1 - 2 baños
7984      1 - 2 baños
7986           1 baño
7994           1 baño
8003           1 baño
8007           1 baño
24471     2 - 4 baños
24472    3 - 12 baños
Name: n_dormitorios, dtype: string

In [317]:
raw.loc[:, "n_dormitorios"] = raw.loc[:, "n_dormitorios"].str.split(pat=" ")

In [318]:
raw.loc[rights_d, "n_dormitorios"] = raw.loc[rights_d, "n_dormitorios"].apply(lambda x: x[:-1]) # Column correction
raw.loc[rights_d, "n_dormitorios"] = raw.loc[rights_d, "n_dormitorios"].apply(lambda x: [x[0],x[-1]] if len(x)==3 else x)

In [319]:
none_b = [np.any([b is None, b is np.nan, b is pd.NA]) for b in raw["n_baños"]]
raw.loc[np.all([none_b,wrong_bath], axis=0), "n_baños"] = raw.loc[np.all([none_b,wrong_bath], axis=0), "n_dormitorios"]
raw.loc[wrong_bath, "n_dormitorios"] = pd.NA
raw.loc[wrong_bath, "n_baños"]

7969      [1, 2]
7970         [1]
7976         [2]
7981         [1]
7983      [1, 2]
7984      [1, 2]
7986         [1]
7994         [1]
8003         [1]
8007         [1]
24471     [2, 4]
24472    [3, 12]
Name: n_baños, dtype: object

### Dropping columns that are redundant

In [320]:
raw = raw.drop(labels=["index","metros_quad"], axis=1)
raw

,oferta,propiedad,monto,moneda,direccion,inmobiliaria,comentario,n_dormitorios,n_baños,comuna,unidad_medida,area
0,venta,departamentos,4990,uf,"comandante malbec 13133, la dehesa, lo barnechea",edificio aires malbec,<NA>,"[1, 3]","[1, 2]",lo barnechea,m²,"[54, 100]"
1,venta,departamentos,4490,uf,"josé manuel infante 1045, barrio italia, provi...",barrio infante,<NA>,"[1, 3]","[1, 2]",providencia,m²,"[39, 79]"
2,venta,departamentos,3065,uf,"edmundo eluchans 3011, reñaca, viña del mar",hoy eluchans,<NA>,"[1, 2]","[1, 2]",viña del mar,m²,"[30, 58]"
3,venta,departamentos,1925,uf,"castellón 1265, barrio poniente, concepción",edificio castellón 1265,<NA>,"[1, 2]","[1, 2]",concepción,m²,"[21, 56]"
4,venta,departamentos,2531,uf,"virginio arias 1371, juan gómez millas, ñuñoa",m ñuñoa,<NA>,"[1, 2]","[1, 2]",ñuñoa,m²,"[31, 51]"
...,...,...,...,...,...,...,...,...,...,...,...,...
33151,arriendo temporal,otros inmuebles,35000,$,"matías cousiño 82, santiago, chile, centro his...",<NA>,<NA>,<NA>,<NA>,santiago,m²,[80]
33152,arriendo temporal,otros inmuebles,10000,$,"santo domingo 191, arauco maipú, maipú",<NA>,arriendo de pieza por dias o mensual,<NA>,<NA>,maipú,m²,[20]
33153,arriendo temporal,otros inmuebles,230000,$,"logroño, san joaquín, chile, metro pedrero, sa...",<NA>,arriendo habitación amoblada a estudiante,<NA>,<NA>,san joaquín,m²,[15]
33154,arriendo temporal,otros inmuebles,380000,$,"av. providencia, providencia, chile, manuel mo...",<NA>,"arriendo pieza,solo a mujeres",<NA>,<NA>,providencia,m²,[4]


In [322]:
raw.to_csv("../data/real_estate_28-06-2022.csv", index=False)